In [9]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection as train_test_split
from tqdm import tqdm

In [20]:
train_data, info = tfds.load('fashion_mnist', split='train', with_info=True)
test_data, info = tfds.load('fashion_mnist', split='test', with_info=True)


In [21]:
# steps 1. reshape to flatten (data is initially 28 * 28), cast from integer to float, normalize to a value between 0 and 1 then return tuple
def format_image(data):
    image = data['image']
    image = tf.reshape(image, [-1])
    image = tf.cast(image, 'float32')
    image = image / 255.0
    return image, data['label']

In [22]:
train_data = train_data.map(format_image)
test_data = test_data.map(format_image)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [23]:
batch_size = 64
train = train_data.shuffle(buffer_size=1024).batch(batch_size)
test = test_data.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
train_acc_metric = tf.keras.metrics.SparseCategoricalA

In [24]:
def base_model():
    inputs = tf.keras.Input(shape=(784,), name='digits')
    x = tf.keras.layers.Dense(64, activation='relu', name='dense_1')(inputs)
    x = tf.keras.layers.Dense(64, activation='relu', name='dense_2')(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax', name='predictions')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [26]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

In [36]:
def apply_gradient(optimizer, model, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = loss_object(y_true=y, y_pred=logits)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    
    return logits, loss

In [37]:
def train_data_for_one_epoch():
    losses = []
    pbar = tqdm(total=len(list(enumerate(train))), position=0, leave=True, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
    for step, (x_batch_train, y_batch_train) in enumerate(train):
        logits, loss_value = apply_gradient(optimizer, model, x_batch_train, y_batch_train)
        losses.append(loss_value)
        pbar.set_description("training for step %s: %.4f", (int(step), float(loss_value)))
        pbar.update()
    return losses

In [38]:
def perform_validation():
    losses = []
    for x_val, y_val in test:
        val_logits = model(x_val)
        val_loss = loss_object(y_true=y_val, y_pred=val_logits)
        losses.append(val_loss)
        val_acc_metric(y_val, val_logits)
    return losses

In [39]:
model = base_model()

epochs = 10
epochs_val_losses, epochs_train_losses = [], []
for epoch in range(epochs):
    print("start of epoch %d", (epoch,))
    
    losses_train = train_data_for_one_epoch()
    train_acc = val_acc_metric.result()
    
    losses_val = perform_validation()
    val_acc = val_acc_metric.result()
    
    losses_train_mean = np.mean(losses_train)
    losses_val_mean = np.mean(losses_val)
    epochs_val_losses = epochs_val_losses.append(losses_val_mean)
    epochs_train_losses = epochs_train_losses.append(losses_train_mean)
    
    train_acc_metric.reset_states()
    val_acc_metric.reset_states()

start of epoch %d (0,)


training for step %s: %.4f: 100%|█████████████████████████████████████████████████████████████████████████████| 938/938


NameError: name 'val_acc_metric' is not defined